# Liskov Substitution Principle (LSP)

## Definition

**If you can use a parent class somewhere, you should be able to use any of its child classes in the same place without breaking anything.**

Think of it like this: if you have a "Vehicle" class and a "Car" class that inherits from it, then anywhere in your code where you expect a Vehicle, you should be able to use a Car instead. The Car should behave like a Vehicle, not surprise you with unexpected behavior.

## What This Means in Simple Terms

### **The Basic Idea:**
When you create a child class (subclass), it should be a "drop-in replacement" for its parent class. If your code works with the parent, it should work exactly the same way with the child.

### **Real-World Analogy:**
Imagine you have a "Pet" class and a "Dog" class that inherits from Pet:
- **Good**: A Dog IS a Pet, so anywhere you need a Pet, a Dog should work perfectly
- **Bad**: A Dog that inherits from Pet but can't eat pet food (breaks the Pet contract)

### **The Key Rule:**
**A child class should do everything the parent class can do, and maybe more, but never less or differently in a surprising way.**

## Why LSP Matters

### **Without LSP (Problems):**
- **Unexpected behavior**: Code breaks when you use child classes
- **Hard to debug**: Things work with parent but fail with child
- **Inheritance confusion**: You can't trust that inheritance actually works
- **Code fragility**: Changes to child classes break existing code

### **With LSP (Benefits):**
- **Reliability**: Code that works with parent classes will work with subclasses
- **Polymorphism**: You can safely use different types interchangeably
- **Maintainability**: Changes to subclasses don't break existing code
- **Design integrity**: Your inheritance relationships make logical sense

## 🚨 Common LSP Violations

### **1. Weakening Preconditions:**
- Parent: "This method works with any positive number"
- Child: "This method only works with numbers between 1-10"
- **Problem**: Child is more restrictive than parent

### **2. Strengthening Postconditions (Side Effects):**
- Parent: "This method returns a number"
- Child: "This method returns a number AND sends an email"
- **Problem**: Child does the same thing PLUS unexpected side effects (sending email)
- **Why it's bad**: The parent promised "just return a number," but child also sends emails

### **3. Changing Expected Behavior (Side Effects):**
- Parent: "This method calculates area"
- Child: "This method calculates area AND changes the shape's size"
- **Problem**: Child does the same thing PLUS unexpected side effects (changing size)
- **Why it's bad**: The parent promised "just calculate area," but child also modifies the object

### **4. Throwing New Exceptions:**
- Parent: "This method never throws exceptions"
- Child: "This method throws exceptions in some cases"
- **Problem**: Child is less reliable than parent

### **⚠️ Important Clarification: "More" vs. "Different"**
When we say "maybe more," we mean **additional helpful features**, not **changing the core behavior**. Here's the difference:

#### **✅ Good "More" (Allowed):**
- **Additional methods**: Child adds new methods that don't interfere with parent's behavior
- **Enhanced functionality**: Child provides extra features while maintaining parent's contract
- **Better performance**: Child does the same thing but faster or more efficiently

#### **❌ Bad "More" (Violates LSP):**
- **Side effects**: Child does the same thing PLUS unexpected side effects
- **Changed behavior**: Child does something different than what the parent promised
- **Broken contracts**: Child violates the parent's promises about what methods do

### ** Concrete Example:**
```python
class Calculator:
    def add(self, a, b):
        return a + b  # Just returns the sum

class SmartCalculator(Calculator):
    def add(self, a, b):
        result = a + b
        print(f"Adding {a} + {b} = {result}")  # ✅ Good "more": helpful logging
        return result

class BadCalculator(Calculator):
    def add(self, a, b):
        result = a + b
        self.send_email("Calculation done!")  # ❌ Bad "more": unexpected side effect
        return result
```

**Why SmartCalculator is OK**: It does the same thing (returns sum) PLUS helpful logging
**Why BadCalculator violates LSP**: It does the same thing (returns sum) PLUS unexpected side effect (sends email)


## ❌ Bad Example (Violates LSP)

Let's look at a classic example that violates the Liskov Substitution Principle:


In [ ]:
# =============================================================================
# ❌ BAD EXAMPLE: Square inheriting from Rectangle (Violates LSP)
# =============================================================================
# This example shows why Square should NOT inherit from Rectangle
# Even though mathematically a square IS a rectangle, in code it breaks LSP

class Rectangle:
    """Parent class: A rectangle with width and height that can be set independently."""
    def __init__(self, width, height):
        self.width = width
        self.height = height
    
    def area(self):
        """Calculate the area of the rectangle."""
        return self.width * self.height
    
    def set_width(self, width):
        """Set the width of the rectangle (height stays the same)."""
        self.width = width
    
    def set_height(self, height):
        """Set the height of the rectangle (width stays the same)."""
        self.height = height
    
    def get_width(self):
        """Get the current width."""
        return self.width
    
    def get_height(self):
        """Get the current height."""
        return self.height

class Square(Rectangle):
    """Child class: A square that inherits from Rectangle."""
    def __init__(self, side):
        # A square is a rectangle, right? Let's inherit.
        # This seems logical at first glance...
        super().__init__(side, side)  # Start with equal width and height
        self.side = side

    def set_width(self, width):
        # ❌ PROBLEM: Square forces width = height
        # When you set width, it also changes height to match
        # This is NOT what Rectangle.set_width() is supposed to do!
        self.width = width
        self.height = width  # ← This is the problem!
        self.side = width

    def set_height(self, height):
        # ❌ PROBLEM: Square forces width = height
        # When you set height, it also changes width to match
        # This is NOT what Rectangle.set_height() is supposed to do!
        self.width = height  # ← This is the problem!
        self.height = height
        self.side = height

# =============================================================================
# FUNCTION THAT EXPECTS A RECTANGLE
# =============================================================================
# This function was written to work with Rectangle objects
# It expects that set_width() and set_height() work independently

def resize_rectangle(rectangle, new_width, new_height):
    """Resize a rectangle to new dimensions."""
    print(f"Resizing rectangle to {new_width} x {new_height}...")
    
    # Step 1: Set the width (height should stay the same)
    rectangle.set_width(new_width)
    print(f"After setting width: {rectangle.get_width()} x {rectangle.get_height()}")
    
    # Step 2: Set the height (width should stay the same)
    rectangle.set_height(new_height)
    print(f"After setting height: {rectangle.get_width()} x {rectangle.get_height()}")
    
    print(f"Final result: {rectangle.get_width()} x {rectangle.get_height()}")
    print(f"Area: {rectangle.area()}")

# =============================================================================
# TESTING: Let's see what happens with each type
# =============================================================================

print("=" * 60)
print("TESTING WITH RECTANGLE (This works as expected):")
print("=" * 60)
rect = Rectangle(5, 3)  # Start with 5x3 rectangle
print(f"Starting rectangle: {rect.get_width()} x {rect.get_height()}")
resize_rectangle(rect, 8, 4)  # Try to resize to 8x4

print("\n" + "=" * 60)
print("TESTING WITH SQUARE (This violates LSP!):")
print("=" * 60)
square = Square(5)  # Start with 5x5 square
print(f"Starting square: {square.get_width()} x {square.get_height()}")
resize_rectangle(square, 8, 4)  # Try to resize to 8x4

# =============================================================================
# WHAT JUST HAPPENED?
# =============================================================================
# With Rectangle: 5x3 → 8x3 → 8x4 (works perfectly)
# With Square:    5x5 → 8x8 → 4x4 (completely wrong!)
#
# The Square broke the expected behavior of Rectangle methods!
# This is a classic LSP violation.
# =============================================================================


Testing with Rectangle:
Rectangle resized to: 8 x 4
Area: 32

Testing with Square (violates LSP!):
Rectangle resized to: 4 x 4
Area: 16


### 🔴 Why is this bad?

The `Square` class violates LSP because:

1. **Unexpected behavior**: When we call `set_width(8)` and `set_height(4)` on a Square, we expect it to become 8x4, but it becomes 4x4
2. **Broken contract**: The `resize_rectangle` function expects to be able to set width and height independently
3. **Not truly substitutable**: A Square cannot replace a Rectangle in all contexts without changing behavior
4. **Violates expectations**: Client code that works with Rectangle will break when given a Square

**Problems:**
- Square changes the behavior of inherited methods in unexpected ways
- Client code cannot rely on the contract established by the parent class
- The inheritance relationship doesn't make logical sense in practice
- Violates the principle of substitutability


## ✅ Good Example (Follows LSP)

Let's fix this by introducing abstractions and dependency injection:

## 🔧 What Are Abstractions? (ABC and abstractmethod)

Before we see the solution, let's understand the tools we'll use:

### **ABC (Abstract Base Class):**
- `ABC` stands for "Abstract Base Class"
- It's a special type of class that cannot be instantiated directly
- Think of it as a "template" or "contract" that other classes must follow
- It's like a blueprint for a house - you can't live in the blueprint, but you can build houses from it

### **abstractmethod:**
- `@abstractmethod` is a decorator that marks a method as "must be implemented"
- Any class that inherits from an ABC with abstract methods MUST implement those methods
- It's like saying "any class that wants to be a Logger MUST have a log() method"


In [ ]:
# =============================================================================
# ✅ GOOD EXAMPLE: Proper LSP Design (Square and Rectangle as separate classes)
# =============================================================================
# This example shows how to design Square and Rectangle to follow LSP
# Key insight: Don't make Square inherit from Rectangle - make them both inherit from Shape

from abc import ABC, abstractmethod

# =============================================================================
# STEP 1: CREATE A COMMON ABSTRACT BASE CLASS
# =============================================================================
# Instead of making Square inherit from Rectangle, we create a common parent: Shape
# This way, both Rectangle and Square can be used wherever a Shape is expected

class Shape(ABC):
    """Abstract base class for all shapes."""
    @abstractmethod
    def area(self):
        """Calculate the area of the shape."""
        pass

# =============================================================================
# STEP 2: CREATE RECTANGLE CLASS (INHERITS FROM SHAPE, NOT ABSTRACT)
# =============================================================================
# Rectangle is a concrete implementation of Shape
# It has its own specific methods for width and height manipulation

class Rectangle(Shape):
    """A rectangle with independent width and height."""
    def __init__(self, width, height):
        self.width = width
        self.height = height
    
    def area(self):
        """Calculate rectangle area: width × height."""
        return self.width * self.height
    
    # Rectangle-specific methods
    def set_width(self, width):
        """Set width (height stays the same)."""
        self.width = width
    
    def set_height(self, height):
        """Set height (width stays the same)."""
        self.height = height
    
    def get_width(self):
        """Get current width."""
        return self.width
    
    def get_height(self):
        """Get current height."""
        return self.height

# =============================================================================
# STEP 3: CREATE SQUARE CLASS (INHERITS FROM SHAPE, NOT RECTANGLE)
# =============================================================================
# Square is also a concrete implementation of Shape
# It has its own specific methods for side manipulation
# This way, Square doesn't break Rectangle's contract!

class Square(Shape):
    """A square with equal sides."""
    def __init__(self, side):
        self.side = side
    
    def area(self):
        """Calculate square area: side × side."""
        return self.side * self.side
    
    # Square-specific methods
    def set_side(self, side):
        """Set the side length."""
        self.side = side
    
    def get_side(self):
        """Get current side length."""
        return self.side

# =============================================================================
# STEP 4: CREATE FUNCTIONS THAT WORK WITH THE ABSTRACT BASE CLASS
# =============================================================================
# These functions can work with ANY Shape (Rectangle, Square, Circle, etc.)
# This is the power of LSP - polymorphism works correctly!

def print_area(shape: Shape):
    """Print the area of any shape."""
    # ✅ This function works with ANY Shape
    # It doesn't care if it's a Rectangle, Square, or any other shape
    print(f"Area: {shape.area()}")

# =============================================================================
# STEP 5: CREATE SPECIFIC FUNCTIONS FOR SPECIFIC SHAPES
# =============================================================================
# These functions work with specific shape types
# They use the specific methods that only those shapes have

def resize_rectangle(rectangle: Rectangle, new_width, new_height):
    """Resize a rectangle to new dimensions."""
    # ✅ This function only works with Rectangle objects
    # It uses Rectangle-specific methods (set_width, set_height)
    rectangle.set_width(new_width)
    rectangle.set_height(new_height)
    print(f"Rectangle resized to: {rectangle.get_width()} x {rectangle.get_height()}")
    print(f"Area: {rectangle.area()}")

def resize_square(square: Square, new_side):
    """Resize a square to new side length."""
    # ✅ This function only works with Square objects
    # It uses Square-specific methods (set_side)
    square.set_side(new_side)
    print(f"Square resized to side: {square.get_side()}")
    print(f"Area: {square.area()}")

# =============================================================================
# STEP 6: TEST THE CORRECTED DESIGN
# =============================================================================
# Now let's see how this design works correctly

print("=" * 60)
print("TESTING RECTANGLE (Works as expected):")
print("=" * 60)
rect = Rectangle(5, 3)
print(f"Starting rectangle: {rect.get_width()} x {rect.get_height()}")
resize_rectangle(rect, 8, 4)  # This works perfectly!

print("\n" + "=" * 60)
print("TESTING SQUARE (Works as expected):")
print("=" * 60)
square = Square(5)
print(f"Starting square: {square.get_side()} x {square.get_side()}")
resize_square(square, 4)  # This also works perfectly!

print("\n" + "=" * 60)
print("TESTING POLYMORPHIC BEHAVIOR (LSP in action!):")
print("=" * 60)
# ✅ This is where LSP shines - we can use different shapes interchangeably
shapes = [Rectangle(3, 4), Square(5)]
for i, shape in enumerate(shapes, 1):
    print(f"Shape {i}: ", end="")
    print_area(shape)  # Works with both Rectangle and Square!

# =============================================================================
# WHY THIS DESIGN FOLLOWS LSP:
# =============================================================================
# 1. ✅ Both Rectangle and Square can be used wherever Shape is expected
# 2. ✅ They both implement the area() method correctly
# 3. ✅ They don't break each other's contracts
# 4. ✅ Polymorphism works correctly
# 5. ✅ No unexpected behavior when substituting one for the other
# =============================================================================


Testing with Rectangle:
Rectangle resized to: 8 x 4
Area: 32

Testing with Square:
Square resized to side: 4
Area: 16

Testing polymorphic behavior:
Area: 12
Area: 25


## Another LSP Example: Bird Classes

Let's look at another common LSP violation with bird classes:


In [3]:
# ❌ Bad Example: Violates LSP
class Bird:
    def fly(self):
        print("Flying...")
    
    def eat(self):
        print("Eating...")

class Eagle(Bird):
    def fly(self):
        print("Eagle is soaring high!")

class Penguin(Bird):
    def fly(self):
        # ❌ Problem: Penguins can't fly!
        raise NotImplementedError("Penguins can't fly!")

# Function that expects any Bird
def make_bird_fly(bird: Bird):
    """Make any bird fly."""
    bird.fly()

# This will work fine
print("Testing Eagle:")
eagle = Eagle()
make_bird_fly(eagle)

# This will crash! ❌
print("\nTesting Penguin:")
penguin = Penguin()
try:
    make_bird_fly(penguin)  # This violates LSP!
except NotImplementedError as e:
    print(f"Error: {e}")
    print("Penguin cannot be substituted for Bird!")


Testing Eagle:
Eagle is soaring high!

Testing Penguin:
Error: Penguins can't fly!
Penguin cannot be substituted for Bird!


In [4]:
# ✅ Good Example: Follows LSP
from abc import ABC, abstractmethod

class Animal(ABC):
    """Base class for all animals."""
    @abstractmethod
    def eat(self):
        pass

class FlyingAnimal(Animal):
    """Base class for animals that can fly."""
    @abstractmethod
    def fly(self):
        pass

class SwimmingAnimal(Animal):
    """Base class for animals that can swim."""
    @abstractmethod
    def swim(self):
        pass

class Eagle(FlyingAnimal):
    def eat(self):
        print("Eagle is eating fish")
    
    def fly(self):
        print("Eagle is soaring high!")

class Penguin(SwimmingAnimal):
    def eat(self):
        print("Penguin is eating fish")
    
    def swim(self):
        print("Penguin is swimming gracefully!")

# Functions that work with specific types
def make_animal_fly(animal: FlyingAnimal):
    """Make a flying animal fly."""
    animal.fly()

def make_animal_swim(animal: SwimmingAnimal):
    """Make a swimming animal swim."""
    animal.swim()

def feed_animal(animal: Animal):
    """Feed any animal."""
    animal.eat()

# Test the corrected design
print("Testing Eagle:")
eagle = Eagle()
make_animal_fly(eagle)
feed_animal(eagle)

print("\nTesting Penguin:")
penguin = Penguin()
make_animal_swim(penguin)
feed_animal(penguin)

print("\nTesting polymorphic behavior:")
animals = [Eagle(), Penguin()]
for animal in animals:
    feed_animal(animal)


Testing Eagle:
Eagle is soaring high!
Eagle is eating fish

Testing Penguin:
Penguin is swimming gracefully!
Penguin is eating fish

Testing polymorphic behavior:
Eagle is eating fish
Penguin is eating fish


### 🟢 Why is this good?

The corrected design follows LSP because:

1. **True substitutability**: Any `FlyingAnimal` can be used wherever a `FlyingAnimal` is expected
2. **Proper inheritance**: Each class only inherits behaviors it can actually perform
3. **No unexpected exceptions**: Subclasses don't throw exceptions that parent classes don't
4. **Consistent behavior**: All subclasses maintain the same contract as their parent classes
5. **Logical relationships**: Inheritance relationships make sense in the real world

**Benefits:**
- ✅ **Reliable polymorphism**: Code works correctly with any subclass
- ✅ **No surprises**: Subclasses behave as expected
- ✅ **Maintainable**: Changes to subclasses don't break existing code
- ✅ **Clear contracts**: Each class has well-defined responsibilities


## Testing LSP Compliance

Let's create tests to verify LSP compliance:


In [5]:
# Test functions to verify LSP compliance
def test_shape_substitutability():
    """Test that all shapes can be used interchangeably for area calculation."""
    shapes = [Rectangle(3, 4), Square(5)]
    
    for shape in shapes:
        area = shape.area()
        assert area > 0, f"Area should be positive, got {area}"
        print(f"✅ {type(shape).__name__} area: {area}")

def test_flying_animal_substitutability():
    """Test that all flying animals can be used interchangeably."""
    flying_animals = [Eagle()]
    
    for animal in flying_animals:
        # This should work without exceptions
        animal.fly()
        animal.eat()
        print(f"✅ {type(animal).__name__} works correctly")

def test_swimming_animal_substitutability():
    """Test that all swimming animals can be used interchangeably."""
    swimming_animals = [Penguin()]
    
    for animal in swimming_animals:
        # This should work without exceptions
        animal.swim()
        animal.eat()
        print(f"✅ {type(animal).__name__} works correctly")

def test_animal_feeding():
    """Test that all animals can be fed (common behavior)."""
    animals = [Eagle(), Penguin()]
    
    for animal in animals:
        # This should work for any animal
        animal.eat()
        print(f"✅ {type(animal).__name__} can be fed")

# Run LSP compliance tests
print("Running LSP compliance tests:\n")
test_shape_substitutability()
print()
test_flying_animal_substitutability()
print()
test_swimming_animal_substitutability()
print()
test_animal_feeding()
print("\n🎉 All LSP tests passed! Classes are properly substitutable.")


Running LSP compliance tests:

✅ Rectangle area: 12
✅ Square area: 25

Eagle is soaring high!
Eagle is eating fish
✅ Eagle works correctly

Penguin is swimming gracefully!
Penguin is eating fish
✅ Penguin works correctly

Eagle is eating fish
✅ Eagle can be fed
Penguin is eating fish
✅ Penguin can be fed

🎉 All LSP tests passed! Classes are properly substitutable.


## Key Takeaways

### Liskov Substitution Principle Summary:

1. **Substitutability**: Subclasses must be able to replace parent classes without breaking functionality
2. **Behavioral consistency**: Subclasses should maintain the same behavior and contracts as parent classes
3. **No unexpected exceptions**: Subclasses shouldn't throw exceptions that parent classes don't
4. **Proper inheritance**: Only inherit when the relationship makes logical sense
5. **Contract preservation**: All methods should work as expected when called on subclasses

### When LSP is violated:
- Subclasses change the behavior of inherited methods in unexpected ways
- Subclasses throw exceptions that parent classes don't
- Client code breaks when using subclasses instead of parent classes
- Inheritance relationships don't make logical sense

### How to ensure LSP compliance:
- **Design by contract**: Clearly define what each method should do
- **Use proper abstractions**: Create interfaces that make logical sense
- **Test substitutability**: Write tests that verify subclasses work in place of parent classes
- **Avoid inheritance when inappropriate**: Sometimes composition is better than inheritance

### Remember:
> "Objects of a superclass should be replaceable with objects of a subclass without altering the correctness of the program" - Barbara Liskov
